This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace.

In [43]:
import os
import codecs
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import pandas as pd
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain

os.environ["OPENAI_API_KEY"] = "voc-11256873311266773806418679d499cb5f2e3.94899606"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

In [44]:
model_name = "gpt-3.5-turbo"
temperature = 0.0
llmOpenAI = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens = 500)

In [45]:
prompt = '''
generate a csv file that store information about {topic} 
make sure to include the following attriutes {attributes}. use a real word examples.
generate {rows} rows

print the data only without intro or end
'''
prompt_template = PromptTemplate(template=prompt, input_variables=['topic', 'attributes', 'rows'])

In [46]:
promt_houses=prompt_template.format(topic="Homes", attributes="Neighborhood, location, bedrooms, bathrooms, house size (m^2), price (k€)", rows="12")
print(promt_houses)


generate a csv file that store information about Homes 
make sure to include the following attriutes Neighborhood, location, bedrooms, bathrooms, house size (m^2), price (k€). use a real word examples.
generate 12 rows

print the data only without intro or end



In [47]:
res = llmOpenAI.invoke(promt_houses)

In [48]:
print(res.content)

Neighborhood,Location,Bedrooms,Bathrooms,House Size (m^2),Price (k€)
West Hollywood,Los Angeles,3,2,150,1200
Upper East Side,New York City,4,3,200,1800
South Beach,Miami,2,1,100,800
Georgetown,Washington D.C.,5,4,300,2500
Pacific Heights,San Francisco,3,2,180,1500
Old Town,Chicago,4,3,220,1900
Beverly Hills,Los Angeles,5,5,400,3500
Back Bay,Boston,2,1,120,1000
Downtown,Seattle,3,2,160,1300
South End,Boston,4,3,220,1800
South Loop,Chicago,2,1,110,900


In [49]:
with codecs.open("houses.csv", "w", "utf-8") as file:
    file.write(res.content)
df = pd.read_csv('houses.csv')
df

,Neighborhood,Location,Bedrooms,Bathrooms,House Size (m^2),Price (k€)
0,West Hollywood,Los Angeles,3,2,150,1200
1,Upper East Side,New York City,4,3,200,1800
2,South Beach,Miami,2,1,100,800
3,Georgetown,Washington D.C.,5,4,300,2500
4,Pacific Heights,San Francisco,3,2,180,1500
5,Old Town,Chicago,4,3,220,1900
6,Beverly Hills,Los Angeles,5,5,400,3500
7,Back Bay,Boston,2,1,120,1000
8,Downtown,Seattle,3,2,160,1300
9,South End,Boston,4,3,220,1800


In [50]:
loader = CSVLoader(file_path='houses.csv')
docs = loader.load()

In [51]:
temperature = 1
llmOpenAI = ChatOpenAI(model_name=model_name, temperature=temperature, max_tokens = 1000)

In [52]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(documents=split_docs, embedding=embeddings)

In [58]:
personal_questions = [   
    "How big do you want your house to be?" 
    "What are 3 most important things for you in choosing this property?", 
    "Which amenities would you like?", 
    "Which transportation options are important to you?",
    "How urban do you want your neighborhood to be?",   
]

personal_answers = answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
]

In [59]:
history = ChatMessageHistory()
history.add_user_message(f"""You are AI sales assisstant that will recommend user a home based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
history.add_ai_message("""Now tell me a summary of a home you're considering in points""")
memory = ConversationSummaryMemory(
    llm=llmOpenAI,
    chat_memory=history,
    memory_key="chat_history", 
    input_key="question",
    buffer=f"The human answered {len(personal_questions)} personal questions. Use them to extract home attributes like location, price, home area and number of rooms",
    return_messages=True)

In [60]:
# create a prompt
prompt=PromptTemplate(
    template="You are an sales assistant who buy homes. Use the following pieces of retrieved context and customer prefrences to provide the customer with information about available home. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's prefernced: {chat_history} \nQuestion: {question}\nAnswer:",
    input_variables=['context', 'chat_history', 'question']
    )
print(prompt)
# create question and answer model to retrieve answers from retrived information
chain_type_kwargs = {'prompt': prompt}

chain = ConversationalRetrievalChain.from_llm(
                                llm=llmOpenAI,
                                chain_type="stuff",
                                retriever=db.as_retriever(),
                                combine_docs_chain_kwargs=chain_type_kwargs,
                                memory=memory
                            )

input_variables=['chat_history', 'context', 'question'] input_types={} partial_variables={} template="You are an sales assistant who buy homes. Use the following pieces of retrieved context and customer prefrences to provide the customer with information about available home. Use five sentences maximum and keep the answer attractive. \nContext: {context} \nCustomer's prefernced: {chat_history} \nQuestion: {question}\nAnswer:"


In [61]:
query = "as a sales assisstant, represent the answer in attractive way"
result = chain.invoke({"question": query})
print(result['answer'])

Imagine coming home to this stunning 3 bedroom, 2 bathroom house located in the vibrant Downtown neighborhood of Seattle. With a spacious house size of 160m², you'll have plenty of room to relax and entertain. And all of this for the amazing price of 1300k€. Don't miss out on this fantastic opportunity to own a beautiful home in the heart of the city!


Second try: 

In [62]:
personal_answers = answers = [
    "A comfortable two-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A energy-efficient house",
    "Bike-friendly roads.",
    "Urban amenities like restaurants and theaters."
]

In [63]:
history = ChatMessageHistory()
history.add_user_message(f"""You are AI sales assisstant that will recommend user a home based on their answers to personal questions. Ask user {len(personal_questions)} questions""")
for i in range(len(personal_questions)):
    history.add_ai_message(personal_questions[i])
    history.add_user_message(personal_answers[i])
    
history.add_ai_message("""Now tell me a summary of a home you're considering in points""")
memory = ConversationSummaryMemory(
    llm=llmOpenAI,
    chat_memory=history,
    memory_key="chat_history", 
    input_key="question",
    buffer=f"The human answered {len(personal_questions)} personal questions. Use them to extract home attributes like location, price, home area and number of rooms",
    return_messages=True)

In [64]:
query = "as a sales assisstant, represent the answer in attractive way"
result = chain.invoke({"question": query})
print(result['answer'])

Step right into your dream home in Downtown Seattle with this stunning 3-bedroom, 2-bathroom house boasting a spacious size of 160m², all for an incredible price of 1300k€. Embrace city living at its finest with this amazing opportunity to own a beautiful home in the heart of the bustling downtown area. Don't wait to make this fabulous property yours!
